In [1]:
%matplotlib inline
import os
import re
import io
import itertools
import pprint

from bokeh.io import output_notebook, show
from bokeh.layouts import gridplot
from bokeh.plotting import figure
from bokeh.models import Range1d, Legend
from bokeh.palettes import all_palettes

import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import xarray as xr

import climatools.lblnew.bestfit_params as bestfits
from climatools.lblnew import setup_bestfit, setup_overlap
import climatools.lblnew.pipeline as pipe_lblnew
import climatools.cliradlw.setup as setup_cliradlw
import climatools.cliradlw.pipeline as pipe_cliradlw

import climatools.html.html as climahtml
from climatools.lblnew.dataio import *
from climatools.plot.plot import *


import IPython.display as display

In [2]:
%run param.py

In [3]:
def load_output_file(path_csv):
    '''
    Load lblnew output .csv file to xarray.Dataset
    
    Parameters
    ----------
    path_csv: str
              Path to the .csv file to be loaded.
    ds: xarray.Dataset
        Data in the input file in the form of an xarray.Dataset.
    '''
    toindex = ['band', 'pressure', 'igg', 'g']    
    df = pd.read_csv(path_csv, sep=r'\s+')
    df = df.set_index([i for i in toindex if i in df.columns])
    df = df.rename(columns={'sfu': 'flug',
                            'sfd': 'fldg',
                            'fnet': 'fnetg',
                            'coolr': 'coolrg'})
    ds = xr.Dataset.from_dataframe(df)

    for l in ('level', 'layer'):
        if l in ds.data_vars:
            if len(ds[l].dims) > 1:
                surface = {d: 0 for d in ds.dims if d != 'pressure'}
                coord_level = ds[l][surface]
                ds.coords[l] = ('pressure', coord_level)
            else:
                ds.coords[l] = ('pressure', ds[l])
    
    return ds

In [4]:
def lblnew_setup(param=None):
    if 'ng_refs' in param:
        return {'setup': setup_bestfit,
                'fname_flux_crd': 'output_flux.dat',
                'fname_cool_crd': 'output_coolr.dat',
                'fname_flux_wgt': 'output_wfluxg.dat',
                'fname_cool_wgt': 'output_wcoolrg.dat'}
    else:
        return {'setup': setup_overlap,
                'fname_flux_crd': 'output_flux.dat',
                'fname_cool_crd': 'output_coolr.dat',
                'fname_flux_wgt': 'output_wflux.dat',
                'fname_cool_wgt': 'output_wcoolr.dat'}

    
def load_lblnew_data(param):
    
    fname_dsname = [('fname_flux_crd', 'ds_flux_crd'),
                    ('fname_cool_crd', 'ds_cool_crd'),
                    ('fname_flux_wgt', 'ds_flux_wgt'),
                    ('fname_cool_wgt', 'ds_cool_wgt')]
    
    d = lblnew_setup(param)
    dir_fortran = pipe_lblnew.get_dir_case(param, setup=d['setup'])
    
    data_dict = {}
    for fname, dsname in fname_dsname:
        fpath = os.path.join(dir_fortran, d[fname])
        data_dict[dsname] = load_output_file(fpath)
    return data_dict
    

    
    

In [5]:
d = load_lblnew_data(PARAM_LBLNEW)

DS_FLUX_CRD = d['ds_flux_crd']
DS_COOL_CRD = d['ds_cool_crd']
DS_FLUX_WGT = d['ds_flux_wgt']
DS_COOL_WGT = d['ds_cool_wgt']

In [6]:
DIR_FORTRAN = pipe_cliradlw.get_fortran_dir(PARAM, 
                                            setup=setup_cliradlw)

PATH_FLUX = os.path.join(DIR_FORTRAN, 'output_flux.dat')
PATH_COOL = os.path.join(DIR_FORTRAN, 'output_coolr.dat')

DS_FLUX = load_output_file(PATH_FLUX)
DS_COOL = load_output_file(PATH_COOL)

In [7]:
output_notebook()

Loading BokehJS ...

In [8]:
def fmt_cool(ds_in):
    ds = ds_in.copy(deep=True)
    if 'igg' in ds.dims:
        ds = ds.sel(igg=1)

    if 'g' in ds.dims:
        ds = ds.sum('g')
            
    if 'band' in ds.dims:
        try:
            ds = ds.squeeze('band')
        except ValueError:
            ds = ds.sum('band')
                
    return ds['coolrg']


def pltdata_cooling_1(ds_cool=None,
                      ds_cool_crd=None, ds_cool_wgt=None):
    


    colors = all_palettes['Set1'][4]
    data = [{'label': 'CLIRAD `coolr_bands`',
             'srs': fmt_cool(ds_cool),
             'line_dash': 'dashed', 'line_width': 5,
             'color': colors[0], 'alpha': .6},
            {'label': 'CRD',
             'srs': fmt_cool(ds_cool_crd),
             'line_dash': 'solid', 'line_width': 1.5,
             'marker': 'circle', 'marker_size': 5,
             'color': colors[2], 'alpha': 1,},
            {'label': 'WGT igg=10 wgt_flux=2',
             'srs': fmt_cool(ds_cool_wgt),
             'line_dash': 'solid', 'line_width': 3,
             'marker': 'square', 'marker_size': 5,
             'color': colors[3], 'alpha': .6}]
    return data


def nice_xlims(pltdata=None, prange=None):
    
    def get_slice(srs):
        return srs.sel(pressure=slice(*prange))
    
    srss = [d['srs'] for d in pltdata]
    vmin = min([get_slice(srs).min() for srs in srss])
    vmax = max([get_slice(srs).max() for srs in srss])
    dv = (vmax - vmin) * .01
    return float(vmin - dv), float(vmax + dv)
    

def plt_cooling_bokeh(pltdata=None):
    '''
    Plot a list of cooling rate profiles using bokeh.
    '''
    ymin = 1e-2 
    ymax = 1020 
        
    p1 = figure(title="Linear pressure scale", 
                plot_width=400)        
    xmin, xmax = nice_xlims(pltdata, prange=(50, 1050))
    for d in pltdata:
        if 'marker' in d:
            getattr(p1, d['marker'])(d['srs'].values, 
                    d['srs'].coords['pressure'].values,
                    color=d['color'], alpha=.7)
        p1.line(d['srs'].values, 
                d['srs'].coords['pressure'].values,
                color=d['color'], alpha=d['alpha'], 
                line_width=d['line_width'], line_dash=d['line_dash'])
    
    p1.y_range = Range1d(ymax, ymin)
    p1.yaxis.axis_label = 'pressure [mb]'   
    p1.x_range = Range1d(xmin, xmax)
    p1.xaxis.axis_label = 'cooling rate [K/day]'
            
    p2 = figure(title='Log pressure scale', y_axis_type='log',
                plot_width=560)
    xmin, xmax = nice_xlims(pltdata, prange=(.01, 200))
    
    rs = []
    for d in pltdata:
        rd = []
        if 'marker' in d:
            r_mark = getattr(p2, d['marker'])(d['srs'].values, 
                        d['srs'].coords['pressure'].values,
                        color=d['color'], alpha=.7)
            rd.append(r_mark)
        r_line = p2.line(d['srs'].values, 
                    d['srs'].coords['pressure'].values,
                    color=d['color'], alpha=d['alpha'], 
                    line_width=d['line_width'], line_dash=d['line_dash'])
        rd.append(r_line)
      
        rs.append(rd)
        
    p2.y_range = Range1d(ymax, ymin)  
    p2.yaxis.axis_label = 'pressure [mb]'
    
    p2.x_range = Range1d(xmin, xmax)
    p2.xaxis.axis_label = 'cooling rate [K/day]'
    
    items = [(d['label'], r) for r, d in zip(rs, pltdata)]
    legend = Legend(items=items, location=(10, -30))
    legend.label_text_font_size = '8pt'
    
    p2.add_layout(legend, 'right')  
    
    show(gridplot(p1, p2, ncols=2, plot_height=500))
    

def script_plt_cooling():
    pltdata = pltdata_cooling_1(
        ds_cool=DS_COOL,
        ds_cool_crd=DS_COOL_CRD,
        ds_cool_wgt=DS_COOL_WGT)
    plt_cooling_bokeh(pltdata=pltdata)
    print('''FIGURE. Cooling rate profiles.''')

In [9]:
def fmt_flux(ds_in, ilevels=None):
    ds = ds_in.copy(deep=True)
    
    for l in ('level', 'layer'):
        if l in ds.data_vars:
            if len(ds[l].dims) > 1:
                surface = {d: 0 for d in ds.dims if d != 'pressure'}
                coord_level = ds[l][surface]
                ds.coords[l] = ('pressure', coord_level)
            else:
                ds.coords[l] = ('pressure', ds[l])
    
    if ilevels:
        ds = ds.isel(pressure=ilevels)
        
    if 'igg' in ds.dims:
        ds = ds.sel(igg=[1])
        ds = ds.squeeze('igg').drop('igg')
    
    if 'g' in ds.dims:
        ds = ds.sum('g')
            
    if 'band' in ds.dims:
        try:
            ds = ds.squeeze('band').drop('band')
        except ValueError:
            ds = ds.sum('band')
                
    df = ds.to_dataframe()
    df = df.set_index('level', append=True)               
    return df


def tbdata_flux(ds_flux=None, ds_flux_crd=None, ds_flux_wgt=None):
    ilevels = [0, 23, -1]
    
    benchmark = {'label': 'CRD',
                 'df': fmt_flux(ds_flux_crd, ilevels=ilevels)}
    
    tbs = [{'label': 'WGT igg=10 wgt_flux=1',
            'df': fmt_flux(ds_flux_wgt, ilevels=ilevels)},
           {'label': 'CLIRAD',
            'df': fmt_flux(ds_flux, ilevels=ilevels)}]
    
    data = {'benchmark': benchmark, 'others': tbs}
    return data


def tb_flux(tbdata=None):
    benchmark = tbdata['benchmark']
    others = tbdata['others']
    
    def show_tb(d=None):
        display.display(d['df'])
        print('Table.', 'Fluxes.', d['label'])
        
    def show_tbdiff(d0=None, d=None):
        df = d['df']
        df.index = d0['df'].index
        display.display(df - d0['df'])
        print('Table.', 'Fluxes.', 
              '({}) - ({})'.format(d['label'], d0['label']))
        
    show_tb(benchmark)
    for d in others:
        show_tb(d)
        
    for d in others:
        show_tbdiff(d=d, d0=benchmark)
        
        
def script_tb_flux():
    data = tbdata_flux(ds_flux=DS_FLUX,
                       ds_flux_crd=DS_FLUX_CRD,
                       ds_flux_wgt=DS_FLUX_WGT)
    tb_flux(tbdata=data)

In [10]:
def band_map():
    '''
    Maps spectral bands in lblnew to spectral bands in clirad.
    '''
    lblnew2clirad = {'1': '1',
            '2': '2',
            '3a': '3',
            '3b': '4',
            '3c': '5',
            '4': '6', 
            '5': '7',
            '6': '8',
            '7': '9', 
            '8': '10',
            '9': '11'}
    
    clirad2lblnew = {clirad: lblnew 
                     for lblnew, clirad in lblnew2clirad.items()}
    return clirad2lblnew


def gasbands():
    bmap = band_map()
    gases = list(PARAM['molecule'].keys())
    
    bands = [bmap[str(b)] for b in PARAM['band']]
    return list(itertools.product(gases, bands))


def print_bestfit_params():
    print('Best-fit values for each (gas, band)')
    print('------------------------------------')
    for gas, band in gasbands():
        print('o {} band{}'.format(gas, band))
        param = bestfits.kdist_params(molecule=gas, band=band)
        for name, value in sorted(param.items()):
            print(2 * ' ' + '{} = {}'.format(name, value))

        
def print_input_param():        
    for name, value in PARAM.items():
        print('{} = {}'.format(name, value))
        
        
def print_lblnew_param():
    for name, value in sorted(PARAM_LBLNEW.items()):
        print('{} = {}'.format(name, value))


In [11]:
'''
Cooling rate profile by layer
'''
def script_coolr_bylayer():
    '''
    Displays cooling rate of each layer
    '''
    keys = ['CRD', 'CLIRAD `coolr_bands`']
    dss = [DS_COOL_CRD, DS_COOL]
    das = [fmt_cool(ds) for ds in dss]
    
    dfs = [da.to_dataframe() for da in das]
    df = pd.concat(dfs, axis=1, keys=keys)
    
    fname = 'coolr_bylayer.csv'
    df.to_csv(fname)
    html_download = '''
    Download: <a href="./{fname}">{fname}</a>'''.format(fname=fname)
    
    with pd.option_context('display.max_rows', 999):
        display.display(df)
        
    display.display(display.HTML(html_download))



In [12]:
'''
Fluxes by layer
'''
def script_flux_bylayer():
    '''
    Display fluxes of each layer
    '''
    keys = ['CRD', 'CLIRAD']
    dss = [DS_FLUX_CRD, DS_FLUX]
    dfs = [fmt_flux(ds) for ds in dss]
    
    df = pd.concat(dfs, axis=1, keys=keys)
    
    fname = 'fluxes_bylayer.csv'
    df.to_csv(fname)
    html_download = '''
    Download: <a href="./{fname}">{fname}</a>'''.format(fname=fname)
    
    with pd.option_context('display.max_rows', 999):
        display.display(df)
        
    display.display(display.HTML(html_download))


In [13]:
def show_html(s):
    display.display(display.HTML(s))

    
def show_markdown(s):
    display.display(display.Markdown(s))

    
def script():
    
    title = '### CLIRAD-LW'
    s_input = 'Input Parameters'
    s_lblnew_param = 'LBLNEW case compaired against'
    s_bestfits = 'Best-fit Parameters'
    s_plt_cooling = 'Cooling Rate Profiles'
    s_tb_flux = 'Flux Comparison'
    s_coolr_bylayer = 'Cooling Rate by Layer'
    s_flux_bylayer = 'Fluxes by Layer'
    
    # title
    show_markdown('### CLIRAD-LW')
    
    # TOC
    show_markdown('### Table of Contents')
    show_html(climahtml.getHTML_hrefanchor(s_input))
    show_html(climahtml.getHTML_hrefanchor(s_lblnew_param))
    show_html(climahtml.getHTML_hrefanchor(s_plt_cooling))
    show_html(climahtml.getHTML_hrefanchor(s_tb_flux))
    show_html(climahtml.getHTML_hrefanchor(s_bestfits))
    show_html(climahtml.getHTML_hrefanchor(s_coolr_bylayer))
    show_html(climahtml.getHTML_hrefanchor(s_flux_bylayer))
    
    # Input parameters
    show_html(climahtml.getHTML_idanchor(s_input))
    show_markdown(climahtml.getMarkdown_sectitle(s_input))
    print_input_param()
    
    # LBLNEW parameters 
    show_html(climahtml.getHTML_idanchor(s_lblnew_param))
    show_markdown(climahtml.getMarkdown_sectitle(s_lblnew_param))
    print_lblnew_param()
    
    # Cooling rate profiles
    show_html(climahtml.getHTML_idanchor(s_plt_cooling))
    show_markdown(climahtml.getMarkdown_sectitle(s_plt_cooling))
    script_plt_cooling()
    
    # Flux comparison
    show_html(climahtml.getHTML_idanchor(s_tb_flux))
    show_markdown(climahtml.getMarkdown_sectitle(s_tb_flux))
    script_tb_flux()
    
    # Best-fit values
    show_html(climahtml.getHTML_idanchor(s_bestfits))
    show_markdown(climahtml.getMarkdown_sectitle(s_bestfits))
    print_bestfit_params()
    
    # Cooling rate by layer
    show_html(climahtml.getHTML_idanchor(s_coolr_bylayer))
    show_markdown(climahtml.getMarkdown_sectitle(s_coolr_bylayer))
    script_coolr_bylayer()
    
    # Fluxes by layer
    show_html(climahtml.getHTML_idanchor(s_flux_bylayer))
    show_markdown(climahtml.getMarkdown_sectitle(s_flux_bylayer))
    script_flux_bylayer()
   
    

In [14]:
script()

### CLIRAD-LW

### Table of Contents

# Input Parameters

atmpro = saw
band = [6]
commitnumber = de5d43e
molecule = {'h2o': 'atmpro'}
tsfc = 257


# LBLNEW case compaired against

...

Best-fit values for each (gas, band)
------------------------------------
o h2o band4
  atmpro = mls
  band = 4
  commitnumber = a06b618
  conc = None
  dv = 0.001
  klin = 1e-24
  molecule = h2o
  ng_adju = [0]
  ng_refs = [3]
  nv = 1000
  option_compute_btable = 0
  option_compute_ktable = 1
  option_wgt_flux = 1
  option_wgt_k = 1
  ref_pts = [(600, 250)]
  tsfc = 294
  vmax = 980
  vmin = 800
  w_diffuse = [(1.66, 1.66, 1.85)]
  wgt = [(0.5, 0.55, 0.85)]


# Cooling Rate by Layer

CRD               CLIRAD `coolr_bands`              
            layer        coolrg                layer        coolrg
pressure                                                          
0.000312      1.0 -4.773080e-06                  NaN           NaN
0.000312      NaN           NaN                  1.0 -6.464412e-05
0.000750      2.0 -4.283837e-06                  2.0 -1.728520e-04
0.001052      3.0 -3.083034e-06                  3.0 -1.987418e-05
0.001476      4.0 -1.523695e-06                  4.0 -1.298243e-05
0.002070      5.0  4.413248e-07                  5.0 -9.578589e-06
0.002904      6.0  2.888268e-06                  6.0 -6.394026e-06
0.004074      7.0  5.902320e-06                  7.0 -3.715881e-06
0.005714      8.0  9.582776e-06                  8.0 -1.062614e-06
0.008015      9.0  1.404460e-05                  9.0  1.642094e-06
0.011243     10.0  1.942057e-05                 10.0  4.623692e-06
0.015771     11.0  2.586223e-05                 11.0  7.871228e-06
0.022122     12.0  3.354533e-05                 12.0  1.163678e-05
0.031031     13.0  4.262457e-05                 13.0  1.747841e-05
0.043528     14.0  5.201964e-05                 14.0  2.236929e-05
0.061057     15.0  6.025742e-05                 15.0  2.478512e-05
0.085645     16.0  6.813618e-05                 16.0  2.763346e-05
0.120136     17.0  7.675202e-05                 17.0  3.143680e-05
0.168516     18.0  8.620178e-05                 18.0  3.547008e-05
0.236378     19.0  9.655763e-05                 19.0  3.969639e-05
0.331549     20.0  1.078836e-04                 20.0  4.422545e-05
0.465100     21.0  1.189622e-04                 21.0  4.986825e-05
0.652400     22.0  1.102238e-04                 22.0  4.678284e-05
0.915100     23.0  7.974316e-05                 23.0  3.304746e-05
1.283650     24.0  5.169624e-05                 24.0  2.064298e-05
1.800600     25.0  3.173575e-05                 25.0  1.192660e-05
2.525700     26.0  1.777655e-05                 26.0  5.872344e-06
3.542800     27.0  8.236996e-06                 27.0  1.735935e-06
4.969550     28.0  2.558256e-06                 28.0 -6.953565e-07
6.970850     29.0 -3.924401e-07                 29.0 -1.939898e-06
9.778100     30.0 -2.199722e-06                 30.0 -2.719073e-06
13.715850    31.0 -3.543780e-06                 31.0 -3.317941e-06
19.239350    32.0 -4.462989e-06                 32.0 -3.761699e-06
26.987250    33.0 -4.727486e-06                 33.0 -3.919285e-06
37.855300    34.0 -4.405250e-06                 34.0 -3.794148e-06
53.100050    35.0 -3.915309e-06                 35.0 -3.728673e-06
73.887500    36.0 -3.378088e-06                 36.0 -3.647816e-06
97.662500    37.0 -2.959760e-06                 37.0 -3.526277e-06
121.437500   38.0 -2.890764e-06                 38.0 -3.558818e-06
145.212500   39.0 -4.144496e-06                 39.0 -5.014494e-06
168.987500   40.0 -5.653625e-06                 40.0 -6.713888e-06
192.762500   41.0 -7.514925e-06                 41.0 -8.794621e-06
216.537500   42.0 -9.167296e-06                 42.0 -1.070862e-05
240.312500   43.0 -1.091409e-05                 43.0 -1.278167e-05
264.087500   44.0 -1.302050e-05                 44.0 -1.521910e-05
287.862500   45.0 -1.513232e-05                 45.0 -1.778936e-05
311.637500   46.0 -1.431798e-05                 46.0 -1.852959e-05
335.412500   47.0 -6.277209e-06                 47.0 -1.428950e-05
359.187500   48.0  1.085454e-05                 48.0 -3.898824e-06
382.962500   49.0  4.840762e-05                 49.0  1.761684e-05
406.737500   50.0  1.335205e-04                 50.0  7.062570e-05
430.512500   51.0  2.921810e-04                 51.0  1.763132e-04
454.287500   52.0  5.562243e-04                 52.0  3.624677e-04
478.062500   53.0  9.577298e-04                 53.0  6.604100e-04
501.837500   54.0  1.525780e-03                 54.0  1.101888e-03
525.612500   55.0  2.281277e-03                 55.0  1.694340e-03
549.387500   56.0  3.230963e-03                 56.0  2.46636

# Fluxes by Layer

CRD                              CLIRAD  \
                         flug          fldg      fnetg       flug   
pressure     level                                                  
0.000000e+00 1     -32.825730  0.000000e+00 -32.825730        NaN   
1.000000e-08 1            NaN           NaN        NaN -32.832936   
6.244000e-04 2     -32.825730  6.857989e-10 -32.825730 -32.832936   
8.759000e-04 3     -32.825730  9.835365e-10 -32.825730 -32.832936   
1.228600e-03 4     -32.825730  1.469212e-09 -32.825730 -32.832936   
1.723400e-03 5     -32.825730  2.262727e-09 -32.825730 -32.832936   
2.417400e-03 6     -32.825730  3.558041e-09 -32.825730 -32.832936   
3.390900e-03 7     -32.825730  5.671116e-09 -32.825730 -32.832936   
4.756500e-03 8     -32.825730  9.115577e-09 -32.825730 -32.832936   
6.672000e-03 9     -32.825730  1.472571e-08 -32.825730 -32.832936   
9.358900e-03 10    -32.825730  2.385628e-08 -32.825730 -32.832936   
1.312780e-02 11    -32.825730  3.870486e-08 -32.825730 -32.832936   
1.841450e-02 12    -32.825730  6.283407e-08 -32.825730 -32.832936   
2.583020e-02 13    -32.825730  1.020134e-07 -32.825730 -32.832936   
3.623230e-02 14    -32.825730  1.655388e-07 -32.825730 -32.832937   
5.082340e-02 15    -32.825730  2.667541e-07 -32.825730 -32.832937   
7.129060e-02 16    -32.825730  4.233210e-07 -32.825730 -32.832937   
1.000000e-01 17    -32.825730  6.622101e-07 -32.825730 -32.832937   
1.402710e-01 18    -32.825730  1.026468e-06 -32.825729 -32.832937   
1.967600e-01 19    -32.825730  1.581729e-06 -32.825729 -32.832937   
2.759970e-01 20    -32.825730  2.427866e-06 -32.825728 -32.832937   
3.871000e-01 21    -32.825730  3.716306e-06 -32.825726 -32.832937   
5.431000e-01 22    -32.825730  5.664122e-06 -32.825724 -32.832937   
7.617000e-01 23    -32.825730  8.240720e-06 -32.825721 -32.832937   
1.068500e+00 24    -32.825729  1.109341e-05 -32.825718 -32.832937   
1.498800e+00 25    -32.825730  1.406823e-05 -32.825716 -32.832937   
2.102400e+00 26    -32.825731  1.716015e-05 -32.825713 -32.832937   
2.949000e+00 27    -32.825732  2.036421e-05 -32.825712 -32.832938   
4.136600e+00 28    -32.825734  2.368084e-05 -32.825711 -32.832940   
5.802500e+00 29    -32.825737  2.725302e-05 -32.825710 -32.832941   
8.139200e+00 30    -32.825742  3.140312e-05 -32.825710 -32.832944   
1.141700e+01 31    -32.825747  3.642775e-05 -32.825711 -32.832947   
1.601470e+01 32    -32.825756  4.258411e-05 -32.825713 -32.832952   
2.246400e+01 33    -32.825767  5.030446e-05 -32.825716 -32.832959   
3.151050e+01 34    -32.825782  6.081029e-05 -32.825721 -32.832969   
4.420010e+01 35    -32.825804  7.640459e-05 -32.825728 -32.832983   
6.200000e+01 36    -32.825836  9.998174e-05 -32.825736 -32.833004   
8.577500e+01 37    -32.825880  1.338970e-04 -32.825746 -32.833034   
1.095500e+02 38    -32.825924  1.697463e-04 -32.825754 -32.833066   
1.333250e+02 39    -32.825969  2.067277e-04 -32.825762 -32.833099   
1.571000e+02 40    -32.826032  2.581524e-04 -32.825774 -32.833146   
1.808750e+02 41    -32.826116  3.260259e-04 -32.825790 -32.833208   
2.046500e+02 42    -32.826225  4.134908e-04 -32.825811 -32.833290   
2.284250e+02 43    -32.826354  5.169134e-04 -32.825837 -32.833390   
2.522000e+02 44    -32.826504  6.365838e-04 -32.825868 -32.833508   
2.759750e+02 45    -32.826680  7.756137e-04 -32.825904 -32.833649   
2.997500e+02 46    -32.826885  9.385803e-04 -32.825947 -32.833816   
3.235250e+02 47    -32.827129  1.141417e-03 -32.825987 -32.834016   
3.473000e+02 48    -32.827421  1.415853e-03 -32.826005 -32.834256   
3.710750e+02 49    -32.827772  1.797203e-03 -32.825974 -32.834545   
3.948500e+02 50    -32.828298  2.460198e-03 -32.825838 -32.834984   
4.186250e+02 51    -32.829124  3.662415e-03 -32.825462 -32.835678   
4.424000e+02 52    -32.830335  5.696166e-03 -32.824639 -32.836693   
4.661750e+02 53    -32.832007  8.934789e-03 -32.823073 -32.838102   
4.899500e+02 54    -32.834195  1.381948e-02 -32.820375 -32.839959   
5.137250e+02 55    -32.8

In [15]:
display.HTML('''<script>
code_show=true; 
function code_toggle() {
 if (code_show){
 $('div.input').hide();
 } else {
 $('div.input').show();
 }
 code_show = !code_show
} 
$( document ).ready(code_toggle);
</script>
<form action="javascript:code_toggle()"><input type="submit" value="Click here to toggle on/off the raw code."></form>''')